In [1]:
import matplotlib
import numpy as np
import os
import torch
import cv2
import gc

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
from torch.autograd import Variable

Репозиторий с SuperGlue склонирован в соседнюю папку

In [3]:
%cd SuperGluePretrainedNetwork/

/home/ichuviliaeva/ocr_course/course_ocr/task1/SuperGluePretrainedNetwork


In [4]:
# from models.superpoint import SuperPoint
from models.utils import read_image
from models.superglue import SuperGlue

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
gpu = torch.device('cuda:0')

In [6]:
DATASET_PATH = Path('/DATA/ichuviliaeva/midv500_compressed/')
assert DATASET_PATH.exists(), DATASET_PATH.absolute()
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

gt_root =  /DATA/ichuviliaeva/midv500_compressed/12_deu_drvlic_new/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/12_deu_drvlic_new/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/33_mac_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/33_mac_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/50_xpo_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/50_xpo_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/38_rou_drvlic/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/38_rou_drvlic/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/27_hrv_passport/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/27_hrv_passport/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/44_ukr_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/44_ukr_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/23_fin_drvlic/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/23_f

(50, course_ocr_t1.data.MidvPackage)

In [24]:
spoint = SuperPoint({'keypoint_threshold': 0.05}).to(gpu)

Loaded SuperPoint model


In [25]:
kp_templates = []
# image_t_templates = []
kp_items = []
# image_t_items = []
Hs = []

In [26]:
for dp in tqdm(data_packs):
    template_img, template_img_t, _ = read_image(
        dp.template_item.img_path,
        device=gpu,
        resize=(int(dp.template_item.image_size[0]), int(dp.template_item.image_size[1])), 
        rotation=0, 
        resize_float=True
    )
        
    kps = spoint({'image': template_img_t})
    for k in kps:
        if type(kps[k]) == torch.Tensor:
            kps[k] = kps[k].detach().cpu()
        else:
            kps[k] = torch.stack(kps[k]).detach().cpu() # запомнить, что сделали stack здесь
                 
    kp_templates.append(kps)
    # image_t_templates.append(template_img_t.detach().cpu())
    
    w, h = dp.template_item.image_size
    h = float(h)
    w = float(w)
    corners = [[0, 0], [w, 0], [w, h], [0, h]]
    
    kp_small = []
    H_small = []
    image_t_small = []
    
    for i in range(len(dp)):
        item_img, item_img_t, _ = read_image(
            dp[i].img_path,
            device=gpu,
            resize=(int(dp[i].image_size[0]), int(dp[i].image_size[1])), 
            rotation=0, 
            resize_float=True
        )
        kps = spoint({'image': item_img_t})
        for k in kps:
            if type(kps[k]) == torch.Tensor:
                kps[k] = kps[k].detach().cpu()
            else:
                kps[k] = torch.stack(kps[k]).detach().cpu() # запомнить, что сделали stack здесь
                 
        kp_small.append(kps)
        # image_t_small.append(item_img_t.detach().cpu())
        
        (H_ideal, mask) = cv2.findHomography(np.array(corners),
                                             np.array(dp[i].gt_data['quad']),
                                             method=cv2.RANSAC)
        H_small.append(H_ideal)    
    
    Hs.append(H_small)         
    kp_items.append(kp_small)
    # image_t_items.append(image_t_small)
    

  0%|                                                                        | 0/50 [00:00<?, ?it/s]

  2%|█▎                                                              | 1/50 [00:07<05:52,  7.19s/it]

 22%|█████████████▊                                                 | 11/50 [01:16<04:26,  6.84s/it]

100%|███████████████████████████████████████████████████████████████| 50/50 [05:38<00:00,  6.76s/it]


In [27]:
print(kp_items[18][287])

{'keypoints': tensor([[[419., 123.],
         [387., 129.],
         [391., 234.],
         [402., 239.],
         [103., 249.],
         [402., 249.],
         [393., 255.],
         [237., 380.],
         [ 73., 546.]]]), 'scores': tensor([[0.0927, 0.1467, 0.0596, 0.0526, 0.0560, 0.0742, 0.0919, 0.0603, 0.0637]]), 'descriptors': tensor([[[-0.1197,  0.0728,  0.0048,  ..., -0.0451,  0.0020, -0.0326],
         [-0.1083,  0.0274, -0.0006,  ...,  0.0458, -0.0924, -0.0311],
         [-0.0532,  0.0674, -0.0186,  ..., -0.0333,  0.0026,  0.1095],
         ...,
         [-0.0530, -0.0383,  0.0602,  ...,  0.0193, -0.0128, -0.0219],
         [ 0.0145, -0.0336,  0.0291,  ..., -0.0275, -0.0469, -0.0378],
         [ 0.1336, -0.0237,  0.0934,  ...,  0.0531, -0.0367,  0.0277]]])}


In [28]:
torch.save(kp_templates, 'kp_templates.pt')
# torch.save(image_t_templates, 'image_t_templates.pt')
torch.save(kp_items, 'kp_items.pt')
# torch.save(image_t_items, 'image_t_items.pt')
torch.save(Hs, 'Hs.pt')

In [7]:
cpu = torch.device('cpu')

In [7]:
kp_templates = torch.load('kp_templates.pt')
kp_items = torch.load('kp_items.pt')
Hs = torch.load('Hs.pt')

In [8]:
sglue = SuperGlue(config={'match_threshold': 0.5}).to(gpu)

Loaded SuperGlue model ("indoor" weights)


In [9]:
def train(optimizer, epochs, heating, lr):
    
    for t in range(epochs):
        sglue.train()
        if t < heating:
            optimizer.lr = lr / 1000
        else:
            optimizer.lr = lr
            
        optimizer.zero_grad()
        
        loss = Variable(torch.tensor(0).float().to(gpu), requires_grad = True)
        
        for i, dp in tqdm(enumerate(data_packs)):
            if i != 0 and i != 1:
            # первые 2 картинки вышли очень большие, при этом менять размер означает и groundtruth пересчитывать
                template_kps = kp_templates[i]
                template_img, template_img_t, _ = read_image(
                    dp.template_item.img_path,
                    device=gpu,
                    resize=(int(dp.template_item.image_size[0]), int(dp.template_item.image_size[1])), 
                    rotation=0, 
                    resize_float=True
                )
                
                # del template_img
                # del _
                
                # gc.collect()
                # torch.cuda.empty_cache()
                
                for k in template_kps:
                    template_kps[k] = Variable(template_kps[k].to(gpu))
                    
                for j in range(len(dp)):
                    if dp[j].is_test_split() == False:
                        item_kps = kp_items[i][j]
                        item_img, item_img_t, _ = read_image(
                            dp[j].img_path,
                            device=gpu,
                            resize=(int(dp[j].image_size[0]), int(dp[j].image_size[1])), 
                            rotation=0, 
                            resize_float=True
                        )
                    
                        # del item_img
                        # del _
                
                        # gc.collect()
                        # torch.cuda.empty_cache()
                    
                        for k in item_kps:
                            item_kps[k] = Variable(item_kps[k].to(gpu))
                        
                        sg_input = dict()
                        for (k, v) in template_kps.items():
                            sg_input[k + '0'] = v
                            sg_input['image0'] = template_img_t
                        
                        for (k, v) in item_kps.items():
                            sg_input[k + '1'] = v
                            sg_input['image1'] = item_img_t
                        
                        sg_output = sglue(sg_input)
                        
                        idx0 = sg_output['matches0'] > -1
                        template_kps_cleared = sg_input['keypoints0'][0][idx0[0]]
                        idx1 = sg_output['matches0'][idx0]
                        item_kps_cleared = sg_input['keypoints1'][0][idx1]
                    
                        loss_part = Variable(torch.tensor(0).float().to(gpu), requires_grad = True)
                        H_ideal = Variable(torch.tensor(Hs[i][j]).float().to(gpu))
                    
                        t_w, t_h = dp[j].image_size
                        for f, p in enumerate(template_kps_cleared):
                            vec = H_ideal @ torch.stack([p[0], p[1], torch.tensor(1.0).to(gpu)])
                            loss_part = loss_part + torch.sqrt(
                                (vec[0] / vec[-1] - item_kps_cleared[f][0])**2 / t_w**2 + \
                                (vec[1] / vec[-1] - item_kps_cleared[f][1])**2 / t_h**2
                            )
                        
                        loss_part = loss_part / max(0.5, template_kps_cleared.shape[0])
                        loss = loss + loss_part
        loss.backward()
        optimizer.step()
        print('epoch ', t, ': loss = ', loss)

In [10]:
optimizer = torch.optim.Adam(sglue.parameters(), lr = 0.001)

In [11]:
train(optimizer, epochs = 10, heating = 2, lr = 0.001)

11it [03:40, 26.57s/it]

50it [19:12, 23.05s/it]


epoch  0 : loss =  tensor(2151.3352, device='cuda:0', grad_fn=<AddBackward0>)


11it [03:08, 22.89s/it]

50it [17:48, 21.37s/it]


...

50it [18:07, 21.75s/it]


epoch  8 : loss =  tensor(2151.3352, device='cuda:0', grad_fn=<AddBackward0>)


11it [03:10, 23.15s/it]

50it [18:02, 21.64s/it]


epoch  9 : loss =  tensor(2151.3352, device='cuda:0', grad_fn=<AddBackward0>)


In [12]:
def eval(refer_item_num, refer_item, test_item_num, test_item):
    sglue.eval()
    template_kps = kp_templates[refer_item_num]
    
    template_img, template_img_t, _ = read_image(
        refer_item.img_path,
        device=gpu,
        resize=(int(refer_item.image_size[0]), int(refer_item.image_size[1])), 
        rotation=0, 
        resize_float=True
    )
    
    for k in template_kps:
        template_kps[k] = template_kps[k].to(gpu)
        
    item_kps = kp_items[refer_item_num][test_item_num]
    
    item_img, item_img_t, _ = read_image(
        refer_item.img_path,
        device=gpu,
        resize=(int(test_item.image_size[0]), int(test_item.image_size[1])), 
        rotation=0, 
        resize_float=True
    )
                        
    for k in item_kps:
        item_kps[k] = item_kps[k].to(gpu)
            
    sg_input = dict()
    for (k, v) in template_kps.items():
        sg_input[k + '0'] = v
        sg_input['image0'] = template_img_t
                        
    for (k, v) in item_kps.items():
        sg_input[k + '1'] = v
        sg_input['image1'] = item_img_t
       
    with torch.no_grad():
        sg_output = sglue(sg_input)
    idx0 = sg_output['matches0'] > -1
    template_kps_cleared = sg_input['keypoints0'][0][idx0[0]]
    idx1 = sg_output['matches0'][idx0]
    item_kps_cleared = sg_input['keypoints1'][0][idx1]
     
    (H, mask) = cv2.findHomography(np.array(template_kps_cleared.detach().cpu()),
                                   np.array(item_kps_cleared.detach().cpu()), 
                                   method=cv2.RANSAC)    
    w, h = refer_item.image_size
    h = float(h)
    w = float(w)
    corners = [[0, 0], [w, 0], [w, h], [0, h]]
    corners_cropped = []
        
    for c in corners:
        vec = H @ np.array([c[0], c[1], 1])
        corners_cropped.append([vec[0] / vec[-1], vec[1] / vec[-1]])
        
    del sg_input
    del sg_output
    
    gc.collect()
    torch.cuda.empty_cache()
    return np.array(corners_cropped) / np.array([w, h])

In [13]:
results_dict = dict()

for i, dp in tqdm(enumerate(data_packs)):
    for j in range(len(dp)):
        if dp[j].is_test_split():
            try:
                results_dict[dp[j].unique_key] = eval(i, dp.template_item, j, dp[j])
            except Exception as exc:
                # Для пропущенных в словаре ключей в метриках автоаматически засчитается IoU=0
                print(exc)

0it [00:00, ?it/s]

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

1it [00:36, 36.41s/it]

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

2it [01:02, 30.37s/it]

...

47it [15:26, 26.03s/it]

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

49it [15:48, 18.40s/it]

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

50it [16:57, 20.36s/it]


In [14]:
from course_ocr_t1.metrics import dump_results_dict, measure_crop_accuracy

In [16]:
%cd ..

/home/ichuviliaeva/ocr_course/course_ocr/task1


In [17]:
dump_results_dict(results_dict, Path() / 'pred.json')

In [18]:
acc = measure_crop_accuracy(
    Path() / 'pred.json',
    Path() / 'gt.json'
)

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.inte

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.inte

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.inte

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.inte

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.inte

In [19]:
print("Точность кропа: {:1.4f}".format(acc))

Точность кропа: 0.0000


Даже после дообучения сеть не начала находить новых матчей. Скорее всего это из-за отвсутствия каких-либо результатов изначально. Теперь она хотя бы не пытается просто ничего не предсказывать.